In [1]:
### 라이브러리 정의하기
# - 모델 또는 파일 저장 라이브러리
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
### 데이터 수집하기
# - 훈련 및 테스트 데이터셋 각각 다운로드하기

In [3]:
### 훈련데이터 다운로드하기
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt",
                           filename = "ratings_train.txt")

('ratings_train.txt', <http.client.HTTPMessage at 0x1d8afc03ca0>)

In [4]:
### 테스트데이터 다운로드하기
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt",
                           filename = "ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x1d8bba2f670>)

In [3]:
### 훈련 및 테스트 데이터 읽어들이기
train_data = pd.read_table("./ratings_train.txt")
test_data = pd.read_table("./ratings_test.txt")

In [4]:
train_data.info()
print("\n")
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [5]:
### 훈련 데이터셋 100개만 가지고 진행
train_data = train_data.iloc[:100, :]
test_data = test_data.iloc[:100, :]
train_data.info()
print("\n")
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        100 non-null    int64 
 1   document  100 non-null    object
 2   label     100 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 2.5+ KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        100 non-null    int64 
 1   document  100 non-null    object
 2   label     100 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 2.5+ KB


In [6]:
### 데이터 정제하기
# - 중복 데이터의 갯수 확인하기
# - 중복되는 데이터가 있다면 duplicate으로 중복 데이터 처리하기
train_data["document"].nunique()

100

In [7]:
### 긍정/참 = 1, 부정/거짓 = 0
train_data["label"].nunique()

2

In [8]:
### 중복데이터 제거하기
# - inplace = True : 실행과 동시에 메모리에 반영시켜라는 뜻
train_data.drop_duplicates(subset=["document"], inplace = True)

In [9]:
len(train_data)

100

In [10]:
### label 데이터에서 긍정/부정의 데이터 빈도를 확인해주세요
train_data.groupby("label").size().reset_index(name="count")

,label,count
0,0,46
1,1,54


In [11]:
### 데이터 정제하기
# - 한글과 공백을 제외하고 모두 제거하기
train_data["document"] = \
    train_data["document"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]",
                                       "",
                                       regex = True)
train_data

,id,document,label
0,9976970,아더빙진짜짜증나네요목소리,0
1,3819312,흠포스터보고초딩영화줄오버연기조차가볍지않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소이야기구먼솔직히재미는없다평점조정,0
4,6483659,사이몬페그의익살스런연기가돋보였던영화스파이더맨에서늙어보이기만했던커스틴던스트가너무나도이...,1
...,...,...,...
95,8763660,어내스트와셀레스틴완전강추에요정말재밌습니다,1
96,9361974,재미있는영화입니다,1
97,7928957,클라라볼라고화신본거아닌데,0
98,10250221,진짜보면서너무슬펐던영화다,1


In [12]:
### 특정행에 공백(스페이스 한칸)이 있는 데이터 조회해서
# - NaN(null)으로 변환한 뒤에
# - 삭제 함수를 사용해서 null인 행 삭제시키기
### 공백이 있는 행을 찾기
train_data["document"] = \
        train_data["document"].str.replace("^ +", "", regex=True)

### 띄어쓰기가 없는 공백들을 모두 NaN 처리하기
train_data["document"].replace("", np.nan, inplace = True)

### 결측치가 있다면, 행 제거하기
train_data = train_data.dropna(how = "any")

### 전체 데이터에서 결측치 확인하기
train_data.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [13]:
### 단어 토큰화
# - 불용어 처리 동시에 진행
okt = Okt()

In [14]:
### 불용어 정의하기
stopwords = ["의", "가", "이", "은", "들", "는", "좀",
             "잘", "걍", "과", "도", "를", "으로", "자",
             "에", "와", "한", "하다"]
print(stopwords)

['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']


In [15]:
### train 단어 토큰화
X_train = []
for sentence in tqdm(train_data["document"]) :
    print(sentence)
    ### 단어 토큰화 : stem -> 어간 단위로 추출을 의미함
    tokenized_sentence = okt.morphs(sentence, stem=True)
    
    ### 불용어 처리하기
    stopwords_removed_sentence = [word
                                  for word in tokenized_sentence
                                  if not word in stopwords]
    
    ### 각 문장별 단어토큰화 결과 리스트에 추가하기
    X_train.append(stopwords_removed_sentence)
    
print(X_train)

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

아더빙진짜짜증나네요목소리


  8%|██████▌                                                                           | 8/100 [00:03<00:29,  3.14it/s]

흠포스터보고초딩영화줄오버연기조차가볍지않구나
너무재밓었다그래서보는것을추천한다
교도소이야기구먼솔직히재미는없다평점조정
사이몬페그의익살스런연기가돋보였던영화스파이더맨에서늙어보이기만했던커스틴던스트가너무나도이뻐보였다
막걸음마뗀세부터초등학교학년생인살용영화ㅋㅋㅋ별반개도아까움
원작의긴장감을제대로살려내지못했다
별반개도아깝다욕나온다이응경길용우연기생활이몇년인지정말발로해도그것보단낫겟다납치감금만반복반복이드라마는가족도없다연기못하는사람만모엿네
액션이없는데도재미있는몇안되는영화
왜케평점이낮은건데꽤볼만한데헐리우드식화려함에만너무길들여져있나
걍인피니트가짱이다진짜짱이다
볼때마다눈물나서죽겠다년대의향수자극허진호는감성절제멜로의달인이다
울면서손들고횡단보도건널때뛰쳐나올뻔이범수연기드럽게못해
담백하고깔끔해서좋다신문기사로만보다보면자꾸잊어버린다그들도사람이었다는것을
취향은존중한다지만진짜내생에극장에서본영화중가장노잼노감동임스토리도어거지고감동도어거지
ㄱ냥매번긴장되고재밋음ㅠㅠ
참사람들웃긴게바스코가이기면락스코라고까고바비가이기면아이돌이라고깐다그냥까고싶어서안달난것처럼보인다


 17%|█████████████▊                                                                   | 17/100 [00:03<00:10,  7.90it/s]

...

절대평범한영화가아닌수작이라는걸말씀드립니다
주제는좋은데중반부터지루하다
다짤랐을꺼야그래서납득할수없었던거야그럴꺼야꼭그랬던걸꺼야
고추를털어버려야할텐데
카밀라벨발연기
재밋는뎅
센스있는연출력탁월한캐스팅년대의향수그래서점
엄포스의위력을다시한번깨닫게해준적남꽃검사님도연기정말좋았어요완전명품드라마
졸쓰레기진부하고말도안됌ㅋㅋ아시간아까워
재밌는데별점이왜이리낮은고
라도기대했던내가죄인입니다죄인입니다
아직도이드라마는내인생의최고
패션에대한열정안나윈투어
키이라나이틀리가연기하고자했던건대체정신장애일까틱장애일까
허허원작가정신나간유령이라재미있겠네요
포스터는있어보이는데관객은명이네
이영화가왜이렇게저평가받는지모르겠다
단순하면서은은한매력의영화
다알바생인가내용도없고무서운거도없고웃긴거도하나도없음완전별싱거운영화ㅇㅇ내ㅇ시간넘아까움완전낚임
오게두어라서리한이굶주렸다
정말맘에들어요그래서또보고싶은데또보는방법이없네ㅜㅡ
윤제문이라는멋진배우를발견하게됐어요소소한일탈이잔잔한미소를머금게합니다음악은조금아쉽네요ㅠㅠ점주고싶은데평점올리고싶어점줄게요
평점에속지마시길시간낭비돈낭비임
리얼리티가뛰어나긴한데큰공감은안간다이민기캐릭터는정신의학상분노조절장애초기증상일거다툭하면사람패고욕하고물건파손하고조금오바였음극초반엔신선했는데가면갈수록이민기정신상태공감불가
마이너스는왜없냐ㅋ뮤비보고영화수준딱알만하더군ㅉㅉ북한에서이런거만들라고돈대주던
난우리영화를사랑합니다
데너리스타르가르엔나도용의주인이되고싶다누이랑근친상간이나하고다닐지라도소설속에선제일멋진놈이자이메라니스터였는데드라마속에선드래곤용이제일멋지네웃음감독님토르다크월드는말아잡수셨을지라도기본선방은했음


 56%|█████████████████████████████████████████████▎                                   | 56/100 [00:04<00:01, 38.55it/s]

영화가사람의영혼을어루만져줄수도있군요거친세상사를잠시잊고동화같은영화에행복했네요
야세르게이작은고추의매운맛을보여주마포퐁저그콩진호가간다
이렇게가슴시리게본드라마가또있을까감동그자체
난또저꼬마애가무슨원한이깊길래했더니그냥혼자나대다걸어쩌라고
재미있어요
전좋아요
최고
너무충격적이엇다기분을완전히푹꺼지게하는느낌활력이라고는하나도없는너무나도무거운지독하고차갑고무자비하다그저일본인들의상상력은정말대단한거같다는생각이든다
심심한영화
백봉기언제나오나요
보는내내그대로들어맞는예측카리스마없는악역
불알이나와서당황아무튼영화가중간에끝나는느낌
평범함속에녹아든평범한일상조금밋밋한게흠
보던거라계속보고있는데전개도느리고주인공인은희는한두컷나오면서소극적인모습에짜증이ㅜㅜ맨날언제끝나나기대만하고있어요전개좀빨리빨리ㅜㅜ
사랑하고싶게하는가슴속온감정을헤집어놓는영화예요정말최고
많은사람들이이다큐를보고우리나라슬픈현대사의한단면에대해깊이생각하고사죄하고바로잡기위해노력했으면합니다말로만듣던보도연맹그민간인학살이이정도일줄이야이건명백한살인입니다살인자들은다어디있나요


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:04<00:00, 21.86it/s]

예전작품캐릭터에피소드재탕삼탕사골우려먹듯우리고내용은산으로가고시청률은아예안나오고이제회중반인데부작이라니
김남길의백점짜리연기력과초반몰입도에도불구하고지루하고손예진ㅈㅈ
재밌네비슷한영화를안보신분들한테는재미있을듯
노래실력으로뽑는게맞냐박시환이나가면진짜망신이다
아일본영화다이런건가유치하다
이틀만에다봤어요재밌어요근데차안에물건넣어조작하려고하면차안이열려있다던지집안이활짝열려서아무나들어간다던가문자를조작하려고하면비번이안걸려있고ㅋㅋㅋ그런건억지스러웠는데그래도내용자체는좋았어요
졸작
재밋네요달팽이가빨라서더재밌었어요
어설픈전개어이없는결말
부패한로마노프왕조를기리는뭣같은영화온몸으로항거했던러시아민중들이그저폭도냐
내용전개는무난한편이였구잘보았습니다
매우실망
한국영화흥행코드갈등갈등계에속갈등화해감동평점점남발흥행뻔하지뭐
아햏햏아햏햏아햏햏
뭐냐시작하고분만에나왔다리플릿사진보며불안하더니만
단연최고라고할수있지
감독이럼먹고영화를만들었나보다관객에게뭘말하는지도모르겠고엉망진창개진창이다
이건뭐냐우뢰매냐
정말쓰레기영화입니다
진정위대한영화최고임
별루였다
내일이기대되는
근데조미가막문위좋아한건가요
ㅋㅋㅋ진짜골깜ㅋㅋ눈부라릴때쓰러짐ㅋㅋ
성룡영화중최악인듯ㅋㅋ
골때리네ㅋㅋㅋㅋ걸스데이이혜리잘되라
서기가이뻐서
완전재밌어요ㅋㅋㅋㅋㅋ백인공주귀여움ㅋㅋㅋㅋㅋㅋ
인상적인영화였다
어내스트와셀레스틴완전강추에요정말재밌습니다
재미있는영화입니다
클라라볼라고화신본거아닌데
진짜보면서너무슬펐던영화다
설정이재밌고새로운에피소드내에서메인스토리도차차나오는게재밌음
[['아더', '빙', '진짜', '짜증나다', '목소리'], ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다'], ['너', '무재', '밓었', '다그', '래서', '보다', '추천', '다'], ['교도소', '이야기', '구먼', '솔직하다', '재미', '없다', '평점', '조정'], ['사이', '몬페', '그', '익살스럽다', '연기', '돋보이다', '영화', '스파이더맨', '에서', '늙다', '보이다', '크다', '스틴던스트

In [16]:
### 테스트 데이터 정제 작업 먼저 진행
# - 한글과 공백을 제외하고 모두 제거하기
test_data["document"] = \
    test_data["document"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]",
                                       "",
                                       regex = True)
test_data

,id,document,label
0,6270596,굳ㅋ,1
1,9274899,,0
2,8544678,뭐야이평점들은나쁘진않지만점짜리는더더욱아니잖아,0
3,6825595,지루하지는않은데완전막장임돈주고보기에는,0
4,6723715,만아니었어도별다섯개줬을텐데왜로나와서제심기를불편하게하죠,0
...,...,...,...
95,6753658,전기톱은못들고다니는데엔진톱이겠죠,0
96,9665771,완전재밌엇는데왜평점이,1
97,8757576,제임스완이내목표임ㄷ,1
98,9850643,점고아깝다개막장영화의원조라고나할까아내와사별한지얼마나지났다고딴여자들만나고다니다가결국맥...,0


In [17]:
### 특정행에 공백(스페이스 한칸)이 있는 데이터 조회해서
# - NaN(null)으로 변환한 뒤에
# - 삭제 함수를 사용해서 null인 행 삭제시키기
### 공백이 있는 행을 찾기
test_data["document"] = \
        test_data["document"].str.replace("^ +", "", regex=True)

### 띄어쓰기가 없는 공백들을 모두 NaN 처리하기
test_data["document"].replace("", np.nan, inplace = True)

### 결측치가 있다면, 행 제거하기
test_data = test_data.dropna(how = "any")

### 전체 데이터에서 결측치 확인하기
test_data.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [18]:
### test 단어 토큰화
X_test = []
for sentence in tqdm(test_data["document"]) :
    print(sentence)
    ### 단어 토큰화 : stem -> 어간 단위로 추출을 의미함
    tokenized_sentence = okt.morphs(sentence, stem=True)
    
    ### 불용어 처리하기
    stopwords_removed_sentence = [word
                                  for word in tokenized_sentence
                                  if not word in stopwords]
    
    ### 각 문장별 단어토큰화 결과 리스트에 추가하기
    X_test.append(stopwords_removed_sentence)
    
print(X_test)

  9%|███████▌                                                                           | 9/99 [00:00<00:01, 69.77it/s]

굳ㅋ
뭐야이평점들은나쁘진않지만점짜리는더더욱아니잖아
지루하지는않은데완전막장임돈주고보기에는
만아니었어도별다섯개줬을텐데왜로나와서제심기를불편하게하죠
음악이주가된최고의음악영화
진정한쓰레기
마치미국애니에서튀어나온듯한창의력없는로봇디자인부터가고개를젖게한다
갈수록개판되가는중국영화유치하고내용없음폼잡다끝남말도안되는무기에유치한남무아그립다동사서독같은영화가이건류아류작이다
이별의아픔뒤에찾아오는새로운인연의기쁨모든사람이그렇지는않네
괜찮네요오랜만포켓몬스터잼밌어요
한국독립영화의한계그렇게아버지가된다와비교됨
청춘은아름답다그아름다움은이성을흔들어놓는다찰나의아름다움을잘포착한섬세하고아름다운수채화같은퀴어영화이다
눈에보이는반전이었지만영화의흡인력은사라지지않았다
스토리연출연기비주얼등영화의기본조차안된영화에무슨평을해이런영화찍고도김문옥감독은내가영화경력이몇인데조무래기들이내영화를평론해같은마인드에빠져있겠지
소위ㅈ문가라는평점은뭐냐
최고
발연기도저히못보겠다진짜이렇게연기를못할거라곤상상도못했네
나이스
별재미도없는거우려먹어챔프에서방송몇번했더라ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
일의금요일나이트메어시리즈와함께가장많은시리즈를양산해냈던헬레이저시리즈의첫편작가의상상력이돋보이는작품이며갈고리로사지찢는고어씬은지금보더라도상당히잔인하고충격적이다


 20%|████████████████▌                                                                 | 20/99 [00:00<00:01, 76.45it/s]

나름교훈돋기는하지만어쩔수없이저평점받을수밖에없는저질섹스코미디
꽤재밌게본영화였다
민주화시대의억눌린영혼의관음적인욕구분출인상적이다
일본천황이미국으로부터받은면죄부의긴박한과정을루즈하고지저분하게늘어놓았다
괜히나올때어미알건드려서긁어부스름만들었다는분이저아래보이던데영화제대로안봤네알이딱까지면서새끼가나오려했음그냥가면그놈한테당했을것임한놈두놈막나올게뻔했으니작살낼수밖에없었다


 47%|██████████████████████████████████████▉                                           | 47/99 [00:00<00:00, 75.37it/s]

번은봤네요어쩜이렇게잘만들었을까
실화라서더욱아름답고찡하네요많이울었어요벌써년이란시간이흘렀네요
이건뭐노답이네년대어린이영화좀보고본받아라
지금까지본영화중마음이가장따뜻해지는영화
너무너무재밌다
안보면후회ㅠㅠ
평점점도주기싫어지는영화배우나감독이라는사람이나영화판에서안봐으면한다그리고평점알바생들너무티난다
정말진짜표현할수없는영화
엉성한액션에시나리오는왜그러지막판에대동단결
은못나올것같다
재미가없어요시간이아깝고
북괴는우리의주적일뿐이다
ㅋㅋㅋ난생처음로그인하고평점남기네요개빡쳐서알바들속지마세요이런ㄱㅐ같은시간낭비가아놔진짜평점마이너스별오만개
셰익스피어는셰익스피어고이영화는이영화
한국영화특선해서봤다개막작선정되서왤까궁금했었는데봐도이율모르겠다
뭐야라는말밖에는
이게영화야
애니는일본이갑인듯
롭코헨의몰락의점
감동적이다
은빛날개의조종사
마음이을보신분들은이해하시겟지만는망장입니다졸작이죠솔직히말해선그이유는예를들면엽기적인그녀는긴시간을두고코메디멜로를그렷습니다하지만이영화는단시간분안에모든것을소화하려는욕심코믹하다분채안돼갑자기생뚱맞게슬픈음악이깔리고
이런스타일액션영화굿굿
제임스헷필드먹고더파워풀해졌어ㅋㅋㅋㅋ
돈만있으면내가이것보단더잘만들겠따ㅋ
레이토와다미앙의시원한액션은어디갔나ㅁㅊ
내참이딴걸드라마라고그냥이건세계보건기구에서발암물질로올려야한다
아극장에서아무기대없이들어갔다가감탄하면서보고나왔는데년도였구나그때가
아이디어가아주좋다재밌다
난재밌던데평점왜케낮지
역사얘기보다영화배우감독들들에게더관심이간다
몇년만에아메리칸조크보고웃어보는거지하아이맛이야바로
재밌다내용도신선하고의미도있으며연기도좋고영상도좋다
여름이면한결같이생각나는드라마


 73%|███████████████████████████████████████████████████████████▋                      | 72/99 [00:00<00:00, 93.56it/s]

콩콩ㅋㄲㅈㅁ
으뭐의도는좋아봤자전달이돼야지
아빠가낚여서본적있다나슈래기영화
결혼은현실이에요징징징성현아벗겨놓고이정도얘기밖에못하냐
엠비씨무비채널에서봤는데또봐도엄청잼나네여
어우이게진짜무서웠어요진짜로시간아까웠습니다
몇번을봐도볼때마다재밌다
재미있게잘봤습니다후속편을기다리게만드네요
결말뻔히알면서보는데도너무슬프고많이울었음
한번본적은업지만재미있을것같다
성경에보면지나치게의인도되지말고지혜롭게되지도말라고했다주인공의지나친착한남자컴플렉스는결국모든걸엉망으로만들었다직장에서해고되고부인에게거부당하고그모든터전을잃을만큼불륜으로생긴아이를보러가는게그리중요했을까
미친놈들집합소네연출력빵점스토리빵점구성빵점
하작은그림보고기대했는데수염눈색깔도뭐이건뭐원작비충실ㄹㄹ
볼거없을때봐도재미없는영화
관객모독관객모독관객모독관객모독
이런내용완전좋다ㅠㅜㅜㅜ
쇼타군넘좋아돈키호테대박재밌음
참대단한것같습니다천재
볼만함
제시카알바가벗고달려드는데쌩까는게말이되냐
솔직히이건급그이하의영화이긴함ㅎㅎ
개성있는몬스터들과귀여운여자아이가만나뿜어내는매력에퐁당빠졌다
제목이찰지구나ㅋㅋㅋ
년작품이라는게믿기지가않을정도로디테일하다안소니퀸의명연기또한물론
아찔한사랑줄다리기
재미있었다또봐야징ㅎ
중간에화면이좀끊기는것빼곤넘좋았어요
이영화를말하는데긴단어는필요없다재수없는졸작이거면충분하다
최고최고바다까지차원고양이로봇도라에몽과함께하는진구와친구들의미지탐험이야기우주꺼지갔음좋겠당
완전잼없음
로코굉장히즐겨보는데이영화는좀별로였다뭔가사랑도개그도억지스런느낌
후속작계획은없나요ㅜ
엔딩때까지음성넣은것은오버의극치다
단두마디감동
뭘만든건가
전기톱은못들고다니는데엔진톱이겠죠
완전재밌엇는데왜평점이
제임스완이내목표임ㄷ
점고아깝다개막장영화의원조라고나할까아내와사별한지얼마나지났다고딴여자들만나고다니다가결국맥라이언한테꽂힌톰행크스나약혼자두고톰행크스랑썸타다가결국엔약혼자버리고톰행크스한테쪼르르달려가는맥라이언이나


100%|██████████████████████████████████████████████████████████████████████████████████| 99/99 [00:01<00:00, 92.96it/s]

가끔문득생각나서다시보는영화색감이정말예술이죠강렬하고화려하고츠지야안나의너무도아름답던리즈시절도볼수있고
[['굳다', 'ㅋ'], ['뭐', '야', '평점', '나쁘다', '않다', '만점', '짜다', '리', '더', '더욱', '아니다'], ['지루하다', '않다', '완전', '막장', '임돈', '주다', '보기', '에는'], ['만', '아니다', '별', '다섯', '개', '주다', '왜', '로나', '서다', '제', '심기', '불편하다'], ['음악', '주가', '되다', '최고', '음악', '영화'], ['진정하다', '쓰레기'], ['마치', '미국', '애니', '에서', '튀어나오다', '창의력', '없다', '로봇', '디자인', '부터가', '고개', '젖다', '다'], ['갈수록', '개판', '되다', '중국영화', '유치하다', '없다', '폼', '잡다', '끝나다', '안되다', '무기', '유치하다', '남무', '아', '그리다', '동사서독', '같다', '영화', '이건', '류', '아', '류작', '이다'], ['이별', '아픔', '뒤', '찾아오다', '새롭다', '인연', '기쁨', '모든', '사람', '그렇다', '않다'], ['괜찮다', '오랜', '만', '포켓몬스터', '잼밌', '어', '요'], ['한국', '독립영화', '한계', '그렇게', '아버지', '되다', '비교', '되다'], ['청춘', '아름답다', '다그', '아름답다', '움', '이성', '을', '흔들다', '찰나', '아름답다', '움', '을', '자다', '포착', '섬세하다', '아름답다', '수채화', '같다', '퀴어', '영화', '이다'], ['눈', '보이다', '반전', '이다', '영화', '흡인', '력', '사라지다', '않다'], ['스토리', '연', '출연', '기비', '주얼', '등', '영화', '기본', '조차', '안되다', '영화', '무슨', '평', '을해', '이런', '

### 정수 인코딩하기

In [19]:
### 정수 인코딩을 위한 객체 생성
tokenizer = Tokenizer()

In [20]:
### 정수 인코딩하기 : 딕셔너리 형태로 만들어짐
# - 빈도를 자동으로 계산해서 빈도가 높은 순서대로 1부터 값을 부여시킴
tokenizer.fit_on_texts(X_train)
print(tokenizer.word_index)

{'영화': 1, '이다': 2, '보다': 3, '없다': 4, '하': 5, '다': 6, '있다': 7, '하고': 8, '재밌다': 9, '연기': 10, '별': 11, '나오다': 12, '정말': 13, '만': 14, '고': 15, '을': 16, '진짜': 17, '사람': 18, '인': 19, '드라마': 20, '재미있다': 21, '좋다': 22, '완전': 23, '내': 24, '최고': 25, '보고': 26, '평점': 27, '그': 28, '에서': 29, '보이다': 30, '되다': 31, '끄다': 32, '저': 33, '아니다': 34, '같다': 35, '게': 36, '그렇다': 37, 'ㅋㅋ': 38, '속': 39, '냐': 40, '않다': 41, '다그': 42, '재미': 43, 'ㅋㅋㅋ': 44, '아깝다': 45, '움': 46, '것': 47, '본': 48, '감동': 49, '임': 50, '라고': 51, '이라': 52, '절대': 53, '지루하다': 54, '음식': 55, '평범하다': 56, '점': 57, '오다': 58, '또': 59, '싶다': 60, '멋지다': 61, '조금': 62, '이나': 63, '적': 64, '전개': 65, '백': 66, '내용': 67, '갈등': 68, '뭐': 69, '아햏햏': 70, '흠': 71, '포스터': 72, '줄': 73, '너': 74, '크다': 75, '너무나도': 76, '이쁘다': 77, '반개': 78, '로': 79, '그것': 80, '반복': 81, '못': 82, '네': 83, '낮다': 84, '데': 85, '너무': 86, '짱': 87, '년대': 88, '향수': 89, '때': 90, '중': 91, '스토리': 92, '어거지': 93, '냥': 94, 'ㅠㅠ': 95, '웃기다': 96, '기면': 97, '갈수록': 98, '말': 99, '깨알': 100, '캐스팅': 101, '위': 

### 단어 집합 확인하기

In [21]:
### 감성분석을 위해서
# - 단어 집합(각 문장)의 크기 확인 필요
# - 단어 집합에서 희귀 단어수 확인 필요
# - 전체 등장 빈도 대비 희귀 단어 등장 비율 확인 필요
# * 희귀단어 정의 : 빈도가 몇보다 작으면 희귀단어로 취급
#                 : 작다라는 기준은 분석가가 정의

In [22]:
### 희귀단어 기준 정의
threshold = 3

# - 전체 단어 수
total_cnt = len(tokenizer.word_index)

### 희귀단어 기준보다 작은 단어의 개수
rare_cnt = 0

### 훈련 데이터의 전체 단어 빈도수 총합
total_freq = 0

### 희귀단어 기준보다 작은 단어의 빈도수의 총합
rare_freq = 0

In [23]:
### 단어 빈도 갯수 확인하기
tokenizer.word_counts

OrderedDict([('아더', 1),
             ('빙', 1),
             ('진짜', 6),
             ('짜증나다', 1),
             ('목소리', 1),
             ('흠', 2),
             ('포스터', 2),
             ('보고', 4),
             ('초딩', 1),
             ('영화', 27),
             ('줄', 2),
             ('오버', 1),
             ('연기', 7),
             ('조차', 1),
             ('가볍다', 1),
             ('않다', 3),
             ('너', 2),
             ('무재', 1),
             ('밓었', 1),
             ('다그', 3),
             ('래서', 1),
             ('보다', 14),
             ('추천', 1),
             ('다', 9),
             ('교도소', 1),
             ('이야기', 1),
             ('구먼', 1),
             ('솔직하다', 1),
             ('재미', 3),
             ('없다', 12),
             ('평점', 4),
             ('조정', 1),
             ('사이', 1),
             ('몬페', 1),
             ('그', 4),
             ('익살스럽다', 1),
             ('돋보이다', 1),
             ('스파이더맨', 1),
             ('에서', 4),
             ('늙다', 1),
             ('보이다', 4),
 

In [24]:
### 단어와 빈도수의 쌍을 key와 value로 받아서 반복 처리하기
for key, value in tokenizer.word_counts.items() :
    total_freq = total_freq + value
    ### 희귀단어 기준보다 작으면
    if value < threshold :
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

In [25]:
print("단어 집합의 크기 : ", total_cnt)
print("희귀 단어 기준보다 작은 단어수 : ", threshold - 1, rare_cnt)
print("단어 집합에서 희귀 단어의 비율 : ", rare_cnt / total_cnt * 100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 : ", rare_freq / total_freq * 100)

단어 집합의 크기 :  705
희귀 단어 기준보다 작은 단어수 :  2 635
단어 집합에서 희귀 단어의 비율 :  90.0709219858156
전체 등장 빈도에서 희귀 단어 등장 빈도 :  66.81943171402384


In [26]:
### 정수인코딩에 적용할 단어의 갯수(사이즈) 결정하기
vocab_size = total_cnt - rare_cnt + 1
print("사이즈 : ", vocab_size)

사이즈 :  71


In [27]:
### 텍스트 시퀀스(문자)를 정수 시퀀스(정수값)으로 변환
tokenizer = Tokenizer(vocab_size)

### 사이즈만큼의 단어들에 대해서 변환하기
tokenizer.fit_on_texts(X_train)

### 훈련 및 테스트 데이터 변환 적용하기
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
print(X_train,"\n")
print(X_test)

[[17], [26, 1, 10, 41], [42, 3, 6], [43, 4, 27], [28, 10, 1, 29, 30, 30], [1, 44, 11, 45, 46], [], [11, 45, 12, 10, 13, 14, 20, 4, 10, 5, 18, 14], [4, 21, 1], [27, 14], [2, 17, 2], [2], [15, 12, 10], [22, 3, 3, 42, 18, 2, 47, 16], [14, 17, 29, 48, 1, 49, 50, 15, 49], [31], [18, 51, 32, 42, 32, 47, 30], [19, 47, 5, 43], [5, 41], [52, 33, 53, 34], [7, 1, 53, 34], [3, 41], [54, 35, 55, 1, 8, 7, 55, 3, 43, 7, 36, 4, 55, 11, 12, 11, 12], [53, 56, 1, 34, 52], [22, 54], [6, 16, 32, 4, 37, 37, 32], [], [], [], [7, 37], [16, 10, 13, 22, 23, 20], [38, 45], [9, 11, 57], [2, 2], [20, 24, 25], [], [10, 5, 2, 2], [52, 21], [7, 30], [33], [1], [6, 24, 4, 4, 4, 23, 11, 1, 24, 46, 23], [58], [13, 59, 26, 60, 59, 3, 4], [61, 5, 36, 36, 62, 15, 60, 27, 57], [27, 50], [6, 6, 18, 15, 8, 8, 62, 2], [4, 26, 1, 14, 29, 51], [1], [19, 31, 63, 39, 61, 2, 20, 39, 61], [1, 18, 16, 7, 35, 1], [16, 33, 28], [36, 48, 20, 59, 7, 49, 28], [33], [21], [22], [25], [64, 16, 5, 4, 13, 35, 6], [1], [12], [3, 4], [63, 1], [

In [28]:
### 훈련 및 테스트 종속변수 생성하기
y_train = np.array(train_data["label"])
y_test = np.array(test_data["label"])
print(y_train, "\n")
print(y_test)

[0 1 0 0 1 0 0 0 1 1 1 1 0 1 0 1 1 0 1 1 1 1 0 1 0 0 1 0 1 1 1 0 1 0 1 1 0
 1 0 1 1 0 1 1 1 0 0 0 0 1 1 0 1 0 1 1 0 1 0 1 0 0 0 0 1 1 0 0 1 0 0 1 0 1
 0 0 1 0 0 1 0 1 0 0 0 1 0 1 1 1 0 1 1 1 1 1 1 0 1 1] 

[1 0 0 0 1 0 0 0 1 1 0 1 1 0 1 1 0 1 0 1 0 1 1 0 1 1 1 0 1 1 1 0 0 0 0 0 0
 0 0 0 0 0 1 0 1 0 0 1 1 0 0 0 1 1 1 0 1 1 1 0 0 0 0 1 0 1 1 1 1 1 0 0 0 0
 1 1 1 1 0 0 1 1 1 0 1 1 0 1 0 0 1 0 1 0 0 1 1 0 1]


In [31]:
### X_train 데이터에 비어있는 데이터 찾아서 삭제하기
# - 길이가 0인 인덱스 삭제하는 방법으로...

### 각 리스트의 길이가 0인 인덱스 찾기
drop_train = [index for index, sentence in enumerate(X_train)
                          if len(sentence) < 1]

### 찾아낸 인덱스 삭제하기
# - 훈련과 독립변수 및 종속변수 모두 동일한 위치의 인덱스 삭제
X_train = np.delete(X_train, drop_train, axis = 0)
y_train = np.delete(y_train, drop_train, axis = 0)

print(len(X_train), len(y_train))

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (100,) + inhomogeneous part.

In [32]:
### 패딩 크기 결정하기
count = 0
max_len = 20
for sentence in X_train :
    if(len(sentence) <= max_len) :
        count = count + 1
print("전체 데이터 중에 길이가 {} 이하인 비율 : {} %".format(
            max_len, (count/len(X_train)) * 100))

전체 데이터 중에 길이가 20 이하인 비율 : 100.0 %


In [33]:
### 모든 단어의 길이를 20으로 맞추기
# - 패딩 처리
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [35]:
# 라이브러리 정의 
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [44]:
# LSTM 모델 생성 및 계층 추가
embedding_dim = 100
hidden_units =128

model = Sequential()

model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1,activation="sigmoid"))

In [52]:
# 콜백 함수 정의
es = EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=4)
mc = ModelCheckpoint("./naver_best_model.h5", monitor="val_loss",  mode="max",verbose=1, save_best_only=True)

In [53]:
# 모델 속성 정의 : compile
model.compile(optimizer="rmsprop",
             loss="binary_crossentropy",
             metrics=["acc"])

In [57]:
# 모델 훈련시키기
history = model.fit(X_train, y_train, epochs=30, callbacks=[es,mc], batch_size=64, validation_split=0.2)

Epoch 1/30
1/2 [==============>...............] - ETA: 0s - loss: 0.3560 - acc: 0.8438
Epoch 1: val_loss did not improve from 0.52501
2/2 [==============================] - 0s 42ms/step - loss: 0.3788 - acc: 0.8125 - val_loss: 0.4213 - val_acc: 0.8000
Epoch 2/30
1/2 [==============>...............] - ETA: 0s - loss: 0.3786 - acc: 0.8125
Epoch 2: val_loss did not improve from 0.52501
2/2 [==============================] - 0s 27ms/step - loss: 0.3788 - acc: 0.8125 - val_loss: 0.3964 - val_acc: 0.8500
Epoch 3/30
1/2 [==============>...............] - ETA: 0s - loss: 0.3615 - acc: 0.7969
Epoch 3: val_loss improved from 0.52501 to 0.61128, saving model to .\naver_best_model.h5
2/2 [==============================] - 0s 55ms/step - loss: 0.3724 - acc: 0.8000 - val_loss: 0.6113 - val_acc: 0.6500
Epoch 4/30
1/2 [==============>...............] - ETA: 0s - loss: 0.3931 - acc: 0.8125
Epoch 4: val_loss did not improve from 0.61128
2/2 [==============================] - 0s 26ms/step - loss: 0.4036 

In [58]:
# 테스트
model.evaluate(X_test,y_test)

4/4 [==============================] - 0s 4ms/step - loss: 0.7195 - acc: 0.6364


[0.7194705605506897, 0.6363636255264282]

In [60]:
# 모델 불러들이기 
loaded_model = load_model('naver_best_model.h5')

In [61]:
# 테스트하기
loaded_model.evaluate(X_test,y_test)

4/4 [==============================] - 0s 4ms/step - loss: 0.7320 - acc: 0.6364


[0.731972873210907, 0.6363636255264282]

In [62]:
# 임의 리뷰 정의
new_sentence = "이 영화 개꿀잼 ㅋㅋㅋ"
# 한글 및 공백 이외 모두 제거
new_sentence = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣]","", new_sentence)
# 단어 토큰화 : 어간(표제어) 기준
new_sentence = okt.morphs(new_sentence, stem=True)
# 불용어 처리
new_sentence = [word for word in new_sentence
               if not word in stopwords]
# 정수인코딩
encoded = tokenizer.texts_to_sequences([new_sentence])
# 패딩 사이즈 처리
pad_new = pad_sequences(encoded, maxlen=max_len)

# 감성 예측(긍정 or 부정)
score =float(loaded_model.predict(pad_new))

if(score>0.5):
    print("{}% 확률로 긍정 리뷰 입니다.".format(score*100))
else:
    print("{}% 확률로 부정 리뷰 입니다.".format(1-score*100))

1/1 [==============================] - 0s 240ms/step
-31.218173146247864% 확률로 부정 리뷰 입니다.


In [ ]:
def sentiment_predict(new_sentence):
    # 임의 리뷰 정의
    # new_sentence = "이 영화 개꿀잼 ㅋㅋㅋ"
    # 한글 및 공백 이외 모두 제거
    new_sentence = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣]","", new_sentence)
    # 단어 토큰화 : 어간(표제어) 기준
    new_sentence = okt.morphs(new_sentence, stem=True)
    # 불용어 처리
    new_sentence = [word for word in new_sentence
                   if not word in stopwords]
    # 정수인코딩
    encoded = tokenizer.texts_to_sequences([new_sentence])
    # 패딩 사이즈 처리
    pad_new = pad_sequences(encoded, maxlen=max_len)

    # 감성 예측(긍정 or 부정)
    score =float(loaded_model.predict(pad_new))

    if(score>0.5):
        print("{}% 확률로 긍정 리뷰 입니다.".format(score*100))
    else:
        print("{}% 확률로 부정 리뷰 입니다.".format(1-score*100))

In [ ]:
sentiment_predict(new_sentence)